In [2]:
import sys
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
import openai
import matplotlib.pyplot as plt
import pandas as pd
import re




[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# read from csv file

df = pd.read_csv('data/synthpersons.csv')
# make it to a list
person_list = df.values.tolist()
for row in person_list:
    # make row to a string
    row = str(row)
    print(row)

['Ola Norman', 45, 'ski tv-titting', 'IT-konsulent', 'Oslo', 'En aktiv og utadvendt person som elsker friluftsliv og teknologi']
['Kari Norman', 43, 'maling lesing', 'sosialarbeider', 'Oslo', 'Elsker å male og lese i sin fritid. Hun er veldig omsorgsfull og hjelper andre gjennom jobben hennes']
['Erik Norman', 18, 'fotball data spill', 'student', 'Oslo', 'En ivrig fotballspiller og gamer. Studerer informatikk på universitetet']
['Marte Norman', 16, 'dans musikk', 'videregående skole', 'Oslo', 'Passionert danser og musikkelsker. Hun er i sitt siste år på videregående skole']
['Jan Norman', 13, 'tegneserie robotikk', 'mellomtrinnet', 'Oslo', 'En kreativ og nysgjerrig gutt som elsker tegneserier og robotikk']
['Lise Norman', 11, 'dyr skriving', 'barneskole', 'Oslo', 'En dyreelsker og begynnende forfatter. Hun drømmer om å bli veterinær']
